# 🏥 Medical Image Segmentation - Results & Evaluation

This notebook evaluates all trained models and creates a comprehensive results summary.

**Prerequisites:** You must have already run:
- `COLAB_TRAIN_UNET.ipynb`
- `COLAB_TRAIN_UNETPP.ipynb`
- `COLAB_TRAIN_TRANSUNET.ipynb`

---

## Step 1: Setup Environment

In [1]:
# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

GPU Available: True
GPU Name: Tesla T4


## Step 2: Clone Repository & Install Dependencies

In [2]:
# Clone the repository
!git clone https://github.com/Prabhat9801/Medical-Image-Segmentation.git
%cd Medical-Image-Segmentation

# Install required packages
!pip install -q timm albumentations opencv-python-headless tqdm

Cloning into 'Medical-Image-Segmentation'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 89 (delta 35), reused 74 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 1.12 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/Medical-Image-Segmentation


## Step 3: Mount Google Drive & Load Results

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import shutil

# Copy all experiment results from Google Drive
results_dir = "/content/drive/MyDrive/medical_segmentation_results"
local_exp_dir = "/content/Medical-Image-Segmentation/experiments"

# Create experiments directory
os.makedirs(local_exp_dir, exist_ok=True)

# Copy UNet results
if os.path.exists(f"{results_dir}/unet_experiments"):
    !cp -r {results_dir}/unet_experiments/* {local_exp_dir}/
    print("✅ UNet results loaded")

# Copy UNet++ results
if os.path.exists(f"{results_dir}/unetpp_experiments"):
    !cp -r {results_dir}/unetpp_experiments/* {local_exp_dir}/
    print("✅ UNet++ results loaded")

# Copy TransUNet results
if os.path.exists(f"{results_dir}/transunet_experiments"):
    !cp -r {results_dir}/transunet_experiments/* {local_exp_dir}/
    print("✅ TransUNet results loaded")

# List all experiments
print("\n📁 Available experiments:")
!ls -la {local_exp_dir}

✅ UNet results loaded
✅ UNet++ results loaded

📁 Available experiments:
total 40
drwxr-xr-x 10 root root 4096 Dec  8 17:08 .
drwxr-xr-x  7 root root 4096 Dec  8 17:04 ..
dr-x------  2 root root 4096 Dec  8 17:05 unet_100pct_20251206_181019
dr-x------  2 root root 4096 Dec  8 17:05 unet_10pct_20251206_175301
dr-x------  2 root root 4096 Dec  8 17:06 unet_25pct_20251206_175631
dr-x------  2 root root 4096 Dec  8 17:06 unet_50pct_20251206_180151
dr-x------  2 root root 4096 Dec  8 17:07 unetpp_100pct_20251206_183240
dr-x------  2 root root 4096 Dec  8 17:07 unetpp_10pct_20251206_180612
dr-x------  2 root root 4096 Dec  8 17:08 unetpp_25pct_20251206_181110
dr-x------  2 root root 4096 Dec  8 17:08 unetpp_50pct_20251206_181916


## Step 4: Extract Data (for evaluation)

In [5]:
# Run the optimized data extraction script
!python colab_extract_data.py

📦 Extracting data with Windows path fix...
✅ Extraction complete!

🔧 Fixing splits.csv paths...
Before fix - sample path: ../data/processed/isic/images\ISIC_0016024.png
After fix - sample path: /content/Medical-Image-Segmentation/data/processed/isic/images/ISIC_0016024.png
✅ splits.csv paths fixed!
✅ Sample files verified - paths are correct!

📊 VERIFICATION
✅ Images extracted: 2594
✅ Masks extracted: 2594
✅ splits.csv: Found

🎉 Data extraction successful!


## Step 5: Evaluate All Models

In [20]:
# Pull the fix
!cd /content/Medical-Image-Segmentation && git pull



remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 1.02 KiB | 1.02 MiB/s, done.
From https://github.com/Prabhat9801/Medical-Image-Segmentation
   2177014..8bc54dc  main       -> origin/main
Updating 2177014..8bc54dc
Fast-forward
 save_all_results.py | 95 ++++++++++++++++++++++++++++++++++++-----------------
 1 file changed, 65 insertions(+), 30 deletions(-)


In [21]:
# Run the automated evaluation script
!python evaluate_all.py

Found 8 experiments to evaluate


Evaluating: unet_100pct_20251206_181019
Model: unet
Checkpoint: /content/Medical-Image-Segmentation/experiments/unet_100pct_20251206_181019/best_model.pt

Running command: /usr/bin/python3 -m src.eval --model unet --checkpoint /content/Medical-Image-Segmentation/experiments/unet_100pct_20251206_181019/best_model.pt --output_dir /content/Medical-Image-Segmentation/experiments/unet_100pct_20251206_181019 --num_vis 8

Using device: cuda
Output directory: /content/Medical-Image-Segmentation/experiments/unet_100pct_20251206_181019/best_model

Loading test dataset...
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow

## Step 6: Generate And Save Results Summary

In [22]:
# Pull the latest script
!cd /content/Medical-Image-Segmentation && git pull

Already up to date.


In [23]:


# Run the results collection script
!python collect_results.py

Found 8 result files

📊 COMPLETE RESULTS SUMMARY
 Model Data Fraction  Dice Score      IoU  Pixel Accuracy
  unet        100pct    0.858863 0.781792        0.948426
  unet         10pct    0.754840 0.649669        0.919165
  unet         25pct    0.801901 0.711594        0.935268
  unet         50pct    0.837719 0.758285        0.944310
unetpp        100pct    0.860755 0.783089        0.949294
unetpp         10pct    0.728808 0.611867        0.904130
unetpp         25pct    0.803797 0.711931        0.934336
unetpp         50pct    0.841458 0.762384        0.945137


✅ Results saved to: /content/Medical-Image-Segmentation/results_summary.csv

Creating comparison plots...
✅ Comparison plots saved to: results_comparison.png

🏆 BEST MODEL OVERALL
Model: unetpp
Data Fraction: 100pct
Dice Score: 0.8608 ± 0.1674
IoU: 0.7831 ± 0.1906
Pixel Accuracy: 0.9493 ± 0.0759
Experiment: unetpp_100pct_20251206_183240

🏆 BEST MODEL PER DATA FRACTION

100pct:
  Model: unetpp
  Dice Score: 0.8608 ± 0.1674
 

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:


# Run the comprehensive results saver
!python save_all_results.py

⚠️  Could not create Drive directory: [Errno 30] Read-only file system: '/content/drive/MyDrive/medical_segmentation_results/final_results'
📊 COLLECTING AND SAVING ALL RESULTS

✅ Found 8 result files

📊 RESULTS SUMMARY
 Model Data Fraction  Dice Score      IoU  Pixel Accuracy
  unet        100pct    0.858863 0.781792        0.948426
  unet         10pct    0.754840 0.649669        0.919165
  unet         25pct    0.801901 0.711594        0.935268
  unet         50pct    0.837719 0.758285        0.944310
unetpp        100pct    0.860755 0.783089        0.949294
unetpp         10pct    0.728808 0.611867        0.904130
unetpp         25pct    0.803797 0.711931        0.934336
unetpp         50pct    0.841458 0.762384        0.945137

✅ CSV saved: /content/Medical-Image-Segmentation/reports/results_summary.csv
✅ Detailed CSV saved: /content/Medical-Image-Segmentation/reports/results_detailed.csv

📊 Creating comparison plots...
✅ Comparison plot saved: /content/Medical-Image-Segmentation/r

In [27]:
import shutil
import os

# Source and destination
local_reports = "/content/Medical-Image-Segmentation/reports"
local_experiments = "/content/Medical-Image-Segmentation/experiments"
drive_dest = "/content/drive/MyDrive/medical_segmentation_final"

# Create destination
os.makedirs(drive_dest, exist_ok=True)

# Copy reports
print("📁 Copying reports...")
shutil.copytree(local_reports, f"{drive_dest}/reports", dirs_exist_ok=True)
print("✅ Reports copied!")

# Copy experiments (this will take a few minutes)
print("\n📁 Copying experiments...")
shutil.copytree(local_experiments, f"{drive_dest}/experiments", dirs_exist_ok=True)
print("✅ Experiments copied!")

print("\n🎉 Everything saved to Google Drive!")
print(f"📍 Location: {drive_dest}")

📁 Copying reports...
✅ Reports copied!

📁 Copying experiments...
✅ Experiments copied!

🎉 Everything saved to Google Drive!
📍 Location: /content/drive/MyDrive/medical_segmentation_final


## 🎉 Evaluation Complete!

**Results saved to:**
- CSV: `results_summary.csv`
- Plots: `results_comparison.png`
- Google Drive: `/content/drive/MyDrive/medical_segmentation_results/final_results/`

**Next steps:**
1. Download results from Google Drive
2. Update your local `reports/report.md`
3. Create final presentation/documentation